# **1. Qiuckstart**

머신러닝에서 자주 수행되는 PyTorch의 API들은 본다



In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

파이토치는 torchvision같은 도메인 특화 라이브러리를 제공하고 이는 다양한 데이터셋을 포함하고 있음

여기서는 FashionMNIST 데이터셋을 사용할거임

Torchvision 데이터셋의 주요인자 2개가있음
1. transform : 입력이미지를 전처리 할때 사용
2. target_transform : 정답을 전처리 할때 사용

In [ ]:
# datasets 은 샘플과 해당 라벨을 저장하는 역할을한다.
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 202kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.79MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.6MB/s]


DataLoader는 dataset 객체를 반복 가능한 객체로 만들어준다.

그러면 배치처리, 샘플링, 셔플, 멀티프로세스를 이용한 데이터로딩을 가능하게함

여기서 배치 크기는 64

In [ ]:
batch_size = 64

# 데이터 로더 생성
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"shape of X [N, C, H, W]: {X.shape}")
  print(f"shaep of y: {y.shape} {y.dtype}")
  break

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shaep of y: torch.Size([64]) torch.int64


# Creating Models

파이토치에서 neural network를 정의하려면 nn.Module을 상속받는 클래스를 생성해야함

- __init__ 함수에서는 신경망의 레이어를 정의

- forward 함수에서는 데이터가 신경망을 통해 어떻게 흐를지 지정

신경망의 연산속도를 높이기 위해 모델을 CUDA 같은 가속기로 이동시키자

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# 모델 정의
# 파이토치 기본 신경망 모듈인 nn.Module을 상속받음
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()     # nn.Module의 초기화 함수도 함께 받음
    self.flatten = nn.Flatten() # 입력이미지를 1D 화
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  # forward 메소드 정의
  def forward(self, x):
    x= self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing the Model Parameters

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

학습할때 입력에 대해 예측을 수행하고 오류를 측정하고 백프랍해 파라미터를 조정할 거임

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset) # 전체 데이터 수
  model.train() # 모델 학습
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) # GPU 로 이동시킴

    # 예측 error를 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 백프랍
    loss.backward()
    optimizer.step() # 파라미터 업데이트
    optimizer.zero_grad() # gradient 초기화

    # 배치 사이즈 100마다 중간 loss를 출력할거임
    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval() # 모델 평가
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item() # pred.argmax(1)이 가장높은 점수 = 예측한 클래스
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: /n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epoch = 5
for t in range(epoch):
  print(f"epoch {t+1}\n------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("끝")

epoch 1
------------------
loss: 2.189254 [   64/60000]
loss: 2.174579 [ 6464/60000]
loss: 2.130856 [12864/60000]
loss: 2.148741 [19264/60000]
loss: 2.092943 [25664/60000]
loss: 2.044686 [32064/60000]
loss: 2.076987 [38464/60000]
loss: 2.004961 [44864/60000]
loss: 2.012915 [51264/60000]
loss: 1.933859 [57664/60000]
Test Error: /n Accuracy: 51.6%, Avg loss: 1.937456 

epoch 2
------------------
loss: 1.973417 [   64/60000]
loss: 1.933092 [ 6464/60000]
loss: 1.837604 [12864/60000]
loss: 1.874814 [19264/60000]
loss: 1.748448 [25664/60000]
loss: 1.708406 [32064/60000]
loss: 1.738116 [38464/60000]
loss: 1.640345 [44864/60000]
loss: 1.663004 [51264/60000]
loss: 1.543388 [57664/60000]
Test Error: /n Accuracy: 59.2%, Avg loss: 1.567188 

epoch 3
------------------
loss: 1.635945 [   64/60000]
loss: 1.583227 [ 6464/60000]
loss: 1.453577 [12864/60000]
loss: 1.516185 [19264/60000]
loss: 1.379888 [25664/60000]
loss: 1.389577 [32064/60000]
loss: 1.402364 [38464/60000]
loss: 1.328007 [44864/60000]
l

# Saving Models

모델을 저장하는 일반적인 방법은 모델 매개변수를 state_dict으로 저장


In [ ]:
# 파라미터만 저장(weights, bias)
# 여기서 model.pth는 저장될 파일명임
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading Models

모델 로드는 모델 구조를 다시 만들고 state dict을 로드하면 됨

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only = True))

<All keys matched successfully>

In [ ]:
# 데이터셋의 class 이름을 정의해줌
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # 예측 수행
x, y = test_data[0][0], test_data[0][1] # 테스트 데이터의 첫번째 샘플의 이미지와 정답임
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y] # 가장 높은 점수를 받은 클래스를 이름으로 바꿈
  print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Ankle boot', Actual: 'Ankle boot'
